In [ ]:
import cv2
import numpy as np

In [ ]:
def getSubImage(src, rect): # получение изображения из повёрнутого бокса
    # Get center, size, and angle from rect
    center, size, theta = rect
    # Convert to int
    center, size = tuple(map(int, center)), tuple(map(int, size))
    # Get rotation matrix for rectangle
    M = cv2.getRotationMatrix2D( center, theta, 1)
    # Perform rotation on src image
    dst = cv2.warpAffine(src, M, src.shape[:2])
    out = cv2.getRectSubPix(dst, size, center)
    return out

In [ ]:
# костыль
if __name__ == '__main__':
    def nothing(*arg):
        pass

cap = cv2.VideoCapture(0)
cv2.namedWindow("result")  # создаем главное окно
crange = [0, 0, 0, 0, 0, 0]

# значения маски, подбираются вручную
h1 = ... * 180 / 360
s1 = ...
v1 = ...
h2 = ... * 180 / 360
s2 = ...
v2 = ...

In [ ]:
# основной цикл захвата и обработки изображения

while True:
    ret, img = cap.read()
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # формируем начальный и конечный цвет фильтра
    h_min = np.array((h1, s1, v1), np.uint8)
    h_max = np.array((h2, s2, v2), np.uint8)

    # накладываем фильтр на кадр в модели HSV
    thresh = cv2.inRange(hsv, h_min, h_max)
    cv2.imshow('mask', thresh)

    result_mask = cv2.bitwise_and(img, img, mask=thresh)

    gray_test_image = cv2.cvtColor(result_mask, cv2.COLOR_BGR2GRAY)
    gray_test_image = cv2.bilateralFilter(gray_test_image, 5, 75, 75)

    # детектирование краёв
    edge = cv2.Canny(gray_test_image, 200, 300, 15)
    contours, hierarchy = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]

    max_id = 0
    for i in range(len(contours)):
        if cv2.contourArea(contours[i]) > cv2.contourArea(contours[max_id]):
            max_id = i

    rect = cv2.minAreaRect(contours[max_id])
    box = np.int0(cv2.boxPoints(rect))
    cv2.drawContours(img, [box], 0, (0, 0, 255), 2)

    cv2.imshow("edge", edge)
    cv2.imshow("camera", img)
    cv2.imshow("result", getSubImage(img, rect))

    if cv2.waitKey(10) == 27:  # Esc
        break
cap.release()
cv2.destroyAllWindows()


![ex1](color.jpg "Пример запуска на разнотонном участке")

![ex1](white.jpg "Пример запуска на схожем оттенке")